# 第五次课作业：

作业一：检索结果的后处理方法

本次作业旨在帮助大家深入掌握：
1. **重排技术（Reranking）**：对初步检索结果进行重新排序，提升相关性。
2. **压缩技术（Compression）**：减少检索结果的冗余，提取关键信息。
3. **校正技术（Correction）**：优化查询本身，改善检索质量。

## 任务一：重排技术（Reranking）

### 1.1 环境准备
安装必要的库，包括向量检索、重排序和语言模型相关工具。

In [20]:
%pip install sentence-transformers chromadb rank-bm25 langchain langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


### 1.2 准备测试数据
创建一个简单的文档集合用于演示后处理技术。

In [1]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

# 准备文档集合
documents = [
    "大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。",
    "ChatGPT 是 OpenAI 开发的对话式 AI 模型，基于 GPT-3.5 和 GPT-4 架构。",
    "向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。",
    "检索增强生成（RAG）结合了信息检索和文本生成，可以提供更准确的答案。",
    "Transformer 架构由 Google 在 2017 年提出，彻底改变了自然语言处理领域。",
    "BERT 是一种双向编码器模型，适用于各种 NLP 任务如文本分类和命名实体识别。",
    "向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性。",
    "深度学习需要大量的训练数据和计算资源，GPU 是常用的加速硬件。",
    "Python 是机器学习和数据科学领域最流行的编程语言之一。",
    "北京是中国的首都，拥有悠久的历史和丰富的文化遗产。"
]

# 初始化 ChromaDB
client = chromadb.Client()
default_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# 创建 collection
collection = client.create_collection(
    name="rag_documents",
    embedding_function=default_ef
)

# 添加文档
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))]
)

print(f"已添加 {len(documents)} 个文档到向量数据库")

/opt/homebrew/Caskroom/miniconda/base/envs/rag-project01/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/homebrew/Caskroom/miniconda/base/envs/rag-project01/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


已添加 10 个文档到向量数据库


### 1.3 基础向量检索
首先执行基础的向量相似度检索，这是后处理的起点。

In [2]:
query = "什么是大语言模型？"

# 执行向量检索，获取前5个结果
results = collection.query(
    query_texts=[query],
    n_results=5
)

print(f"查询: {query}\n")
print("初始检索结果（向量相似度排序）：")
for i, (doc, distance) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"{i+1}. [距离: {distance:.4f}] {doc[:50]}...")

查询: 什么是大语言模型？

初始检索结果（向量相似度排序）：
1. [距离: 0.7484] 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。...
2. [距离: 0.8587] 向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性。...
3. [距离: 0.8939] 向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。...
4. [距离: 1.0178] 检索增强生成（RAG）结合了信息检索和文本生成，可以提供更准确的答案。...
5. [距离: 1.0254] 北京是中国的首都，拥有悠久的历史和丰富的文化遗产。...


### 1.4 跨编码器重排（Cross-Encoder Reranking）
使用 Cross-Encoder 模型对初步检索结果进行重新打分和排序。Cross-Encoder 同时处理查询和文档，可以捕捉更深层的语义关系。

In [3]:
from sentence_transformers import CrossEncoder
import numpy as np

# 加载 Cross-Encoder 模型
rerank_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# 获取初步检索的文档
candidate_docs = results['documents'][0]

# 构建查询-文档对
pairs = [[query, doc] for doc in candidate_docs]

# 使用 Cross-Encoder 重新打分
rerank_scores = rerank_model.predict(pairs)

# 按重排分数排序
reranked_indices = np.argsort(rerank_scores)[::-1]

print("\n重排后的结果（Cross-Encoder 排序）：")
for rank, idx in enumerate(reranked_indices):
    print(f"{rank+1}. [重排分数: {rerank_scores[idx]:.4f}] {candidate_docs[idx][:50]}...")


重排后的结果（Cross-Encoder 排序）：
1. [重排分数: 7.5419] 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。...
2. [重排分数: 6.9363] 向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。...
3. [重排分数: 6.5408] 向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性。...
4. [重排分数: 6.3104] 检索增强生成（RAG）结合了信息检索和文本生成，可以提供更准确的答案。...
5. [重排分数: 1.6864] 北京是中国的首都，拥有悠久的历史和丰富的文化遗产。...


### 1.5 基于多样性的重排
除了相关性，我们还可以考虑结果的多样性，避免返回过于相似的文档。使用 MMR（Maximal Marginal Relevance）算法。

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def maximal_marginal_relevance(query_embedding, doc_embeddings, doc_texts, lambda_param=0.5, k=3):
    """
    MMR 算法实现
    lambda_param: 控制相关性和多样性的权重 (0-1)
                 1 = 只考虑相关性，0 = 只考虑多样性
    """
    selected_indices = []
    remaining_indices = list(range(len(doc_embeddings)))
    
    # 计算所有文档与查询的相似度
    query_similarities = cosine_similarity(
        [query_embedding], 
        doc_embeddings
    )[0]
    
    while len(selected_indices) < k and remaining_indices:
        mmr_scores = []
        
        for idx in remaining_indices:
            # 相关性分数
            relevance = query_similarities[idx]
            
            # 多样性分数（与已选文档的最大相似度）
            if selected_indices:
                selected_embeddings = [doc_embeddings[i] for i in selected_indices]
                diversity = max(cosine_similarity(
                    [doc_embeddings[idx]], 
                    selected_embeddings
                )[0])
            else:
                diversity = 0
            
            # MMR 分数
            mmr_score = lambda_param * relevance - (1 - lambda_param) * diversity
            mmr_scores.append(mmr_score)
        
        # 选择 MMR 分数最高的文档
        best_idx_pos = np.argmax(mmr_scores)
        best_idx = remaining_indices[best_idx_pos]
        selected_indices.append(best_idx)
        remaining_indices.remove(best_idx)
    
    return selected_indices

# 获取查询和文档的嵌入
model = SentenceTransformer('all-MiniLM-L6-v2')
query_emb = model.encode(query)
doc_embs = model.encode(candidate_docs)

# 应用 MMR
mmr_indices = maximal_marginal_relevance(
    query_emb, 
    doc_embs, 
    candidate_docs, 
    lambda_param=0.7,
    k=3
)

print("\nMMR 重排结果（平衡相关性和多样性）：")
for rank, idx in enumerate(mmr_indices):
    print(f"{rank+1}. {candidate_docs[idx][:50]}...")


MMR 重排结果（平衡相关性和多样性）：
1. 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。...
2. 向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性。...
3. 向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。...


## 任务二：压缩技术（Compression）

### 2.1 基于相关性的过滤
设置相似度阈值，过滤掉不够相关的文档。

In [ ]:
def filter_by_relevance(documents, scores, threshold=0.3):
    """
    根据相关性分数过滤文档
    """
    filtered = []
    for doc, score in zip(documents, scores):
        if score >= threshold:
            filtered.append((doc, score))
    return filtered

# 应用相关性过滤
# 注意：distance 越小越相似，这里转换为相似度分数
similarity_scores = [1 - d for d in results['distances'][0]]
filtered_results = filter_by_relevance(
    candidate_docs, 
    similarity_scores, 
    threshold=0.2
)

print(f"原始结果数量: {len(candidate_docs)}")
print(f"过滤后结果数量: {len(filtered_results)}")
print("\n过滤后的文档：")
for i, (doc, score) in enumerate(filtered_results):
    print(f"{i+1}. [相似度: {score:.4f}] {doc[:50]}...")

原始结果数量: 5
过滤后结果数量: 1

过滤后的文档：
1. [相似度: 0.2516] 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。...


### 2.2 上下文压缩（Contextual Compression）
使用 LangChain 的上下文压缩器，从检索到的文档中提取与查询最相关的部分。

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# 创建文档对象
docs = [Document(page_content=doc) for doc in candidate_docs]

# 创建嵌入过滤器
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings_filter = EmbeddingsFilter(
    embeddings=embeddings,
    similarity_threshold=0.5
)

# 手动压缩文档
compressed_docs = embeddings_filter.compress_documents(docs, query)

print(f"\n压缩前文档数量: {len(docs)}")
print(f"压缩后文档数量: {len(compressed_docs)}")
print("\n压缩后的文档：")
for i, doc in enumerate(compressed_docs):
    print(f"{i+1}. {doc.page_content[:60]}...")

/var/folders/y5/mq71659d69b2_wlmhhvy7lw00000gn/T/ipykernel_25942/253050070.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/opt/homebrew/Caskroom/miniconda/base/envs/rag-project01/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



压缩前文档数量: 5
压缩后文档数量: 3

压缩后的文档：
1. 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。...
2. 向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性。...
3. 向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。...


### 2.3 文本摘要压缩
对于长文档，可以使用摘要技术提取关键信息，减少 token 消耗。

In [ ]:
def simple_compression(text, max_length=50):
    """
    简单的文本压缩：截取前 N 个字符
    实际应用中可以使用 LLM 进行智能摘要
    """
    if len(text) <= max_length:
        return text
    return text[:max_length] + "..."

print("文本压缩示例：\n")
for i, doc in enumerate(candidate_docs[:3]):
    compressed = simple_compression(doc, max_length=30)
    print(f"原文 {i+1}: {doc}")
    print(f"压缩 {i+1}: {compressed}")
    print(f"压缩率: {len(compressed)/len(doc)*100:.1f}%\n")

文本压缩示例：

原文 1: 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。
压缩 1: 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥...
压缩率: 78.6%

原文 2: 向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性。
压缩 2: 向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性...
压缩率: 106.5%

原文 3: 向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。
压缩 3: 向量数据库是专门用于存储和检索高维向量的数据库系统，在 RA...
压缩率: 78.6%



## 任务三：校正技术（Correction）

### 3.1 查询扩展（Query Expansion）
通过添加同义词或相关词来扩展查询，提高召回率。

In [ ]:
def expand_query(query, synonyms_dict):
    """
    简单的查询扩展
    """
    expanded_terms = [query]
    
    for word, synonyms in synonyms_dict.items():
        if word in query:
            expanded_terms.extend(synonyms)
    
    return " ".join(expanded_terms)

# 定义同义词字典
synonyms = {
    "大语言模型": ["LLM", "大模型", "语言模型"],
    "检索": ["搜索", "查询", "查找"],
}

original_query = "什么是大语言模型？"
expanded_query = expand_query(original_query, synonyms)

print(f"原始查询: {original_query}")
print(f"扩展查询: {expanded_query}")

# 使用扩展查询进行检索
expanded_results = collection.query(
    query_texts=[expanded_query],
    n_results=3
)

print("\n扩展查询的检索结果：")
for i, doc in enumerate(expanded_results['documents'][0]):
    print(f"{i+1}. {doc[:60]}...")

原始查询: 什么是大语言模型？
扩展查询: 什么是大语言模型？ LLM 大模型 语言模型

扩展查询的检索结果：
1. 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。...
2. 向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性。...
3. 向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。...


### 3.2 查询重写（Query Rewriting）
将用户的自然语言查询转换为更适合检索的形式。在实际应用中，可以使用 LLM 进行智能重写。

In [ ]:
def rewrite_query(query):
    """
    简单的查询重写规则
    实际应用中应使用 LLM 进行智能重写
    """
    # 移除疑问词
    question_words = ["什么是", "如何", "怎么", "为什么", "？", "?"]
    rewritten = query
    
    for word in question_words:
        rewritten = rewritten.replace(word, "")
    
    # 去除多余空格
    rewritten = " ".join(rewritten.split())
    
    return rewritten

# 测试查询重写
test_queries = [
    "什么是大语言模型？",
    "如何使用向量数据库？",
    "为什么 RAG 系统很重要？"
]

print("查询重写示例：\n")
for q in test_queries:
    rewritten = rewrite_query(q)
    print(f"原始: {q}")
    print(f"重写: {rewritten}\n")

查询重写示例：

原始: 什么是大语言模型？
重写: 大语言模型

原始: 如何使用向量数据库？
重写: 使用向量数据库

原始: 为什么 RAG 系统很重要？
重写: RAG 系统很重要



### 3.3 混合检索策略（Hybrid Search）
结合关键词检索（BM25）和向量检索，提高检索的准确性和召回率。

In [ ]:
%pip install jieba

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 38.1 MB/s  0:00:00 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314509 sha256=3ea04589bcc7ac54f58fab7e5c6a8cde2a1d83b1c599360f2e56cbfd3569c501
  Stored in directory: /Users/lipucheng/Library/Caches/pip/wheels/32/98/ba/a37fcadb96c75c8f9366a3d17da29cdf8a745ffd38d0092e0d
Successfully built jieba
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from rank_bm25 import BM25Okapi
import jieba

def hybrid_search(query, documents, vector_weight=0.5, k=5):
    """
    混合检索：结合 BM25 和向量检索
    """
    # BM25 检索
    tokenized_corpus = [list(jieba.cut(doc)) for doc in documents]
    bm25 = BM25Okapi(tokenized_corpus)
    tokenized_query = list(jieba.cut(query))
    bm25_scores = bm25.get_scores(tokenized_query)
    
    # 向量检索（使用之前的结果）
    vector_results = collection.query(
        query_texts=[query],
        n_results=len(documents)
    )
    
    # 归一化分数
    bm25_scores_norm = (bm25_scores - bm25_scores.min()) / (bm25_scores.max() - bm25_scores.min() + 1e-10)
    
    # 向量距离转换为相似度分数并归一化
    vector_distances = results['distances'][0]
    vector_scores = np.array([1 - d for d in vector_distances])
    vector_scores_norm = (vector_scores - vector_scores.min()) / (vector_scores.max() - vector_scores.min() + 1e-10)
    
    # 混合分数（只对前5个文档计算）
    hybrid_scores = []
    for i in range(min(len(documents), len(vector_scores_norm))):
        hybrid_score = vector_weight * vector_scores_norm[i] + (1 - vector_weight) * bm25_scores_norm[i]
        hybrid_scores.append((i, hybrid_score))
    
    # 排序
    hybrid_scores.sort(key=lambda x: x[1], reverse=True)
    
    return hybrid_scores[:k]

# 执行混合检索
hybrid_results = hybrid_search(
    query="大语言模型",
    documents=candidate_docs,
    vector_weight=0.6,
    k=3
)

print("混合检索结果（BM25 + 向量检索）：\n")
for rank, (idx, score) in enumerate(hybrid_results):
    print(f"{rank+1}. [混合分数: {score:.4f}] {candidate_docs[idx][:60]}...")

/opt/homebrew/Caskroom/miniconda/base/envs/rag-project01/lib/python3.11/site-packages/jieba/_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/y5/mq71659d69b2_wlmhhvy7lw00000gn/T/jieba.cache
Loading model cost 0.311 seconds.
Prefix dict has been built successfully.


混合检索结果（BM25 + 向量检索）：

1. [混合分数: 1.0000] 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。...
2. [混合分数: 0.3612] 向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性。...
3. [混合分数: 0.2849] 向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。...


## 任务四：综合应用示例

### 4.1 完整的检索后处理流程
组合多种后处理技术，构建一个完整的检索增强管道。

In [ ]:
def advanced_retrieval_pipeline(query, top_k=3):
    """
    高级检索管道：
    1. 查询重写
    2. 初始检索（多召回）
    3. 重排序（Cross-Encoder）
    4. 压缩（相关性过滤）
    5. 多样性优化（MMR）
    """
    print("="*80)
    print("高级检索管道流程")
    print("="*80)
    
    # 步骤1: 查询重写
    rewritten_query = rewrite_query(query)
    print(f"\n步骤1 - 查询重写:")
    print(f"  原始查询: {query}")
    print(f"  重写查询: {rewritten_query}")
    
    # 步骤2: 初始检索（召回更多候选）
    initial_results = collection.query(
        query_texts=[rewritten_query],
        n_results=10
    )
    candidates = initial_results['documents'][0]
    print(f"\n步骤2 - 初始检索: 召回 {len(candidates)} 个候选文档")
    
    # 步骤3: 重排序
    pairs = [[rewritten_query, doc] for doc in candidates]
    rerank_scores = rerank_model.predict(pairs)
    reranked_indices = np.argsort(rerank_scores)[::-1]
    print(f"\n步骤3 - 重排序: 使用 Cross-Encoder 重新排序")
    
    # 步骤4: 相关性过滤
    threshold = 0.0  # Cross-Encoder 分数阈值
    filtered_candidates = []
    filtered_scores = []
    for idx in reranked_indices:
        if rerank_scores[idx] >= threshold:
            filtered_candidates.append(candidates[idx])
            filtered_scores.append(rerank_scores[idx])
    
    print(f"\n步骤4 - 相关性过滤: 保留 {len(filtered_candidates)} 个高相关文档")
    
    # 步骤5: 多样性优化（MMR）
    if len(filtered_candidates) > top_k:
        doc_embs = model.encode(filtered_candidates)
        query_emb = model.encode(rewritten_query)
        mmr_indices = maximal_marginal_relevance(
            query_emb,
            doc_embs,
            filtered_candidates,
            lambda_param=0.7,
            k=top_k
        )
        final_docs = [filtered_candidates[i] for i in mmr_indices]
        final_scores = [filtered_scores[i] for i in mmr_indices]
    else:
        final_docs = filtered_candidates[:top_k]
        final_scores = filtered_scores[:top_k]
    
    print(f"\n步骤5 - 多样性优化: 返回 {len(final_docs)} 个最终结果")
    
    # 输出最终结果
    print("\n" + "="*80)
    print("最终检索结果")
    print("="*80)
    for i, (doc, score) in enumerate(zip(final_docs, final_scores)):
        print(f"\n{i+1}. [分数: {score:.4f}]")
        print(f"   {doc}")
    
    return final_docs

# 测试完整管道
test_query = "什么是大语言模型？"
final_results = advanced_retrieval_pipeline(test_query, top_k=3)

高级检索管道流程

步骤1 - 查询重写:
  原始查询: 什么是大语言模型？
  重写查询: 大语言模型

步骤2 - 初始检索: 召回 10 个候选文档

步骤3 - 重排序: 使用 Cross-Encoder 重新排序

步骤4 - 相关性过滤: 保留 10 个高相关文档

步骤5 - 多样性优化: 返回 3 个最终结果

最终检索结果

1. [分数: 8.1558]
   大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。

2. [分数: 6.7359]
   向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。

3. [分数: 6.2852]
   向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性。


### 4.2 对比不同方法的效果
比较基础检索、重排序和完整管道的检索效果。

In [ ]:
def compare_retrieval_methods(query):
    """
    对比不同检索方法
    """
    print(f"查询: {query}")
    print("\n" + "="*80)
    
    # 方法1: 基础向量检索
    basic_results = collection.query(
        query_texts=[query],
        n_results=3
    )
    print("\n方法1: 基础向量检索")
    print("-" * 80)
    for i, doc in enumerate(basic_results['documents'][0]):
        print(f"{i+1}. {doc[:70]}...")
    
    # 方法2: 向量检索 + Cross-Encoder 重排
    candidates = collection.query(query_texts=[query], n_results=5)['documents'][0]
    pairs = [[query, doc] for doc in candidates]
    scores = rerank_model.predict(pairs)
    reranked_idx = np.argsort(scores)[::-1][:3]
    
    print("\n方法2: 向量检索 + Cross-Encoder 重排")
    print("-" * 80)
    for i, idx in enumerate(reranked_idx):
        print(f"{i+1}. [分数: {scores[idx]:.4f}] {candidates[idx][:60]}...")
    
    # 方法3: 完整管道（已在上面运行）
    print("\n方法3: 完整后处理管道")
    print("-" * 80)
    print("(参见上面的完整管道输出)")
    
    print("\n" + "="*80)

# 运行对比
compare_retrieval_methods("大语言模型的应用")

查询: 大语言模型的应用


方法1: 基础向量检索
--------------------------------------------------------------------------------
1. 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。...
2. 向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。...
3. 向量嵌入可以将文本转换为数值向量，使计算机能够理解语义相似性。...

方法2: 向量检索 + Cross-Encoder 重排
--------------------------------------------------------------------------------
1. [分数: 8.2927] 大语言模型（LLM）是一种基于深度学习的自然语言处理模型，拥有数十亿甚至数千亿参数。...
2. [分数: 7.5426] 向量数据库是专门用于存储和检索高维向量的数据库系统，在 RAG 系统中扮演重要角色。...
3. [分数: 7.2078] 检索增强生成（RAG）结合了信息检索和文本生成，可以提供更准确的答案。...

方法3: 完整后处理管道
--------------------------------------------------------------------------------
(参见上面的完整管道输出)



## 作业2：Sakila Text2SQL 评估体系

### 2.1 任务背景
Text2SQL 是将自然语言问题转换为 SQL 查询的任务。本作业将构建一个评估体系，用于评估 Text2SQL 系统的性能。

我们将使用 Sakila 数据库的问题-SQL对作为评估数据集，实现基于向量检索的 Few-Shot 示例选择，并评估生成的 SQL 质量。

### 2.2 加载评估数据集
首先加载 Sakila Text2SQL 数据集。

In [4]:
import json
import pandas as pd
from pathlib import Path

# 加载数据集
data_path = Path('sample_data/q2sql_pairs.json')
with open(data_path, 'r', encoding='utf-8') as f:
    text2sql_data = json.load(f)

# 转换为 DataFrame 便于分析
df = pd.DataFrame(text2sql_data)

print(f"数据集大小: {len(df)} 条")
print(f"\n数据集概览:")
print(df.head())

# 分析数据集
print("\n数据集统计:")
print(f"- 平均问题长度: {df['question'].str.len().mean():.1f} 字符")
print(f"- 平均 SQL 长度: {df['sql'].str.len().mean():.1f} 字符")

# 分析 SQL 类型分布
sql_types = df['sql'].str.extract(r'^(SELECT|INSERT|UPDATE|DELETE)')[0].value_counts()
print("\nSQL 类型分布:")
for sql_type, count in sql_types.items():
    print(f"  {sql_type}: {count} ({count/len(df)*100:.1f}%)")

数据集大小: 36 条

数据集概览:
                                            question  \
0          List all actors with their IDs and names.   
1                  Add a new actor named 'John Doe'.   
2  Update the last name of actor with ID 1 to 'Sm...   
3                        Delete the actor with ID 2.   
4             Show all films and their descriptions.   

                                                 sql  
0  SELECT actor_id, first_name, last_name FROM ac...  
1  INSERT INTO actor (first_name, last_name) VALU...  
2  UPDATE actor SET last_name = 'Smith' WHERE act...  
3              DELETE FROM actor WHERE actor_id = 2;  
4      SELECT film_id, title, description FROM film;  

数据集统计:
- 平均问题长度: 39.5 字符
- 平均 SQL 长度: 59.7 字符

SQL 类型分布:
  SELECT: 9 (25.0%)
  INSERT: 9 (25.0%)
  UPDATE: 9 (25.0%)
  DELETE: 9 (25.0%)


### 2.3 构建向量检索系统
使用向量检索为给定问题找到最相似的示例，用于 Few-Shot 学习。

In [5]:
import chromadb
from chromadb.utils import embedding_functions

# 创建 Text2SQL 示例库
text2sql_client = chromadb.Client()
text2sql_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# 创建 collection
text2sql_collection = text2sql_client.create_collection(
    name="text2sql_examples",
    embedding_function=text2sql_ef
)

# 添加所有示例（将问题和SQL组合作为文档）
documents = []
metadatas = []
ids = []

for idx, row in df.iterrows():
    # 使用问题作为检索的主要内容
    doc = row['question']
    documents.append(doc)
    metadatas.append({
        'question': row['question'],
        'sql': row['sql']
    })
    ids.append(f"example_{idx}")

text2sql_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print(f"已添加 {len(documents)} 个 Text2SQL 示例到向量库")

已添加 36 个 Text2SQL 示例到向量库


### 2.4 实现示例检索功能
为新问题检索最相关的 Few-Shot 示例。

In [6]:
def retrieve_examples(question, n_examples=3):
    """
    为给定问题检索最相关的示例
    """
    results = text2sql_collection.query(
        query_texts=[question],
        n_results=n_examples
    )
    
    examples = []
    for i, metadata in enumerate(results['metadatas'][0]):
        examples.append({
            'question': metadata['question'],
            'sql': metadata['sql'],
            'distance': results['distances'][0][i]
        })
    
    return examples

# 测试示例检索
test_question = "Find all actors whose first name is 'John'."
retrieved_examples = retrieve_examples(test_question, n_examples=3)

print(f"问题: {test_question}\n")
print("检索到的相似示例:\n")
for i, ex in enumerate(retrieved_examples):
    print(f"{i+1}. [距离: {ex['distance']:.4f}]")
    print(f"   问题: {ex['question']}")
    print(f"   SQL: {ex['sql']}\n")

问题: Find all actors whose first name is 'John'.

检索到的相似示例:

1. [距离: 0.5495]
   问题: List all actors with their IDs and names.
   SQL: SELECT actor_id, first_name, last_name FROM actor;

2. [距离: 0.7341]
   问题: Add a new actor named 'John Doe'.
   SQL: INSERT INTO actor (first_name, last_name) VALUES ('John', 'Doe');

3. [距离: 0.9341]
   问题: Update the last name of actor with ID 1 to 'Smith'.
   SQL: UPDATE actor SET last_name = 'Smith' WHERE actor_id = 1;



### 2.5 构建 Few-Shot Prompt
基于检索到的示例构建 Few-Shot 提示词。

In [7]:
def build_few_shot_prompt(question, examples, n_shots=3):
    """
    构建 Few-Shot 提示词
    """
    prompt = """You are an expert SQL query generator for the Sakila database.
Given a natural language question, generate the corresponding SQL query.

Here are some examples:

"""
    
    # 添加示例
    for i, ex in enumerate(examples[:n_shots]):
        prompt += f"Example {i+1}:\n"
        prompt += f"Question: {ex['question']}\n"
        prompt += f"SQL: {ex['sql']}\n\n"
    
    # 添加新问题
    prompt += f"Now generate the SQL query for this question:\n"
    prompt += f"Question: {question}\n"
    prompt += f"SQL:"
    
    return prompt

# 测试 Few-Shot Prompt 构建
test_question = "Show all films released in 2006."
examples = retrieve_examples(test_question, n_examples=3)
prompt = build_few_shot_prompt(test_question, examples, n_shots=3)

print("构建的 Few-Shot Prompt:\n")
print("="*80)
print(prompt)
print("="*80)

构建的 Few-Shot Prompt:

You are an expert SQL query generator for the Sakila database.
Given a natural language question, generate the corresponding SQL query.

Here are some examples:

Example 1:
Question: Show all films and their descriptions.
SQL: SELECT film_id, title, description FROM film;

Example 2:
Question: Show inventory items for film ID 5.
SQL: SELECT inventory_id, film_id, store_id FROM inventory WHERE film_id = 5;

Example 3:
Question: List all actors with their IDs and names.
SQL: SELECT actor_id, first_name, last_name FROM actor;

Now generate the SQL query for this question:
Question: Show all films released in 2006.
SQL:


### 2.6 评估指标实现

实现常用的 Text2SQL 评估指标：
1. **精确匹配（Exact Match）**: SQL 语句完全相同
2. **标准化匹配（Normalized Match）**: 忽略大小写和空格
3. **关键词匹配（Keyword Match）**: 检查关键 SQL 关键词是否匹配

In [8]:
import re
from typing import Dict, List

def normalize_sql(sql: str) -> str:
    """
    标准化 SQL 查询
    """
    # 转为小写
    sql = sql.lower()
    # 移除多余空格
    sql = ' '.join(sql.split())
    # 移除末尾分号
    sql = sql.rstrip(';')
    return sql

def exact_match(predicted: str, ground_truth: str) -> bool:
    """
    精确匹配评估
    """
    return predicted.strip() == ground_truth.strip()

def normalized_match(predicted: str, ground_truth: str) -> bool:
    """
    标准化匹配评估
    """
    return normalize_sql(predicted) == normalize_sql(ground_truth)

def extract_sql_keywords(sql: str) -> set:
    """
    提取 SQL 中的关键词和表名
    """
    sql_normalized = normalize_sql(sql)
    # 提取主要关键词
    keywords = set()
    
    # SQL 操作类型
    for op in ['select', 'insert', 'update', 'delete', 'from', 'where', 'join', 'order by', 'group by']:
        if op in sql_normalized:
            keywords.add(op)
    
    # 提取表名 (简单的启发式方法)
    # FROM 后面的词通常是表名
    from_pattern = r'from\s+(\w+)'
    from_matches = re.findall(from_pattern, sql_normalized)
    keywords.update(from_matches)
    
    # UPDATE 后面的词通常是表名
    update_pattern = r'update\s+(\w+)'
    update_matches = re.findall(update_pattern, sql_normalized)
    keywords.update(update_matches)
    
 # INSERT INTO 后面的词通常是表名
    insert_pattern = r'insert\s+into\s+(\w+)'
    insert_matches = re.findall(insert_pattern, sql_normalized)
    keywords.update(insert_matches)
    
    return keywords

def keyword_match_score(predicted: str, ground_truth: str) -> float:
    """
    关键词匹配分数（Jaccard 相似度）
    """
    pred_keywords = extract_sql_keywords(predicted)
    gt_keywords = extract_sql_keywords(ground_truth)
    
    if not pred_keywords and not gt_keywords:
        return 1.0
    if not pred_keywords or not gt_keywords:
        return 0.0
    
    intersection = pred_keywords & gt_keywords
    union = pred_keywords | gt_keywords
    
    return len(intersection) / len(union)

def evaluate_sql(predicted: str, ground_truth: str) -> Dict[str, any]:
    """
    综合评估一个 SQL 预测
    """
    return {
        'exact_match': exact_match(predicted, ground_truth),
        'normalized_match': normalized_match(predicted, ground_truth),
        'keyword_score': keyword_match_score(predicted, ground_truth)
    }

# 测试评估指标
print("评估指标测试:\n")

test_cases = [
    {
        'predicted': 'SELECT actor_id, first_name, last_name FROM actor;',
        'ground_truth': 'SELECT actor_id, first_name, last_name FROM actor;',
        'description': '完全匹配'
    },
    {
        'predicted': 'select actor_id, first_name, last_name from actor',
        'ground_truth': 'SELECT actor_id, first_name, last_name FROM actor;',
        'description': '大小写不同，无分号'
    },
    {
        'predicted': 'SELECT * FROM actor;',
        'ground_truth': 'SELECT actor_id, first_name, last_name FROM actor;',
        'description': '列名不同'
    },
    {
        'predicted': 'SELECT * FROM film;',
        'ground_truth': 'SELECT actor_id, first_name, last_name FROM actor;',
        'description': '表名不同'
    }
]

for i, test in enumerate(test_cases):
    print(f"测试 {i+1}: {test['description']}")
    print(f"  预测: {test['predicted']}")
    print(f"  真实: {test['ground_truth']}")
    
    results = evaluate_sql(test['predicted'], test['ground_truth'])
    print(f"  结果:")
    print(f"    - 精确匹配: {results['exact_match']}")
    print(f"    - 标准化匹配: {results['normalized_match']}")
    print(f"    - 关键词分数: {results['keyword_score']:.2f}")
    print()

评估指标测试:

测试 1: 完全匹配
  预测: SELECT actor_id, first_name, last_name FROM actor;
  真实: SELECT actor_id, first_name, last_name FROM actor;
  结果:
    - 精确匹配: True
    - 标准化匹配: True
    - 关键词分数: 1.00

测试 2: 大小写不同，无分号
  预测: select actor_id, first_name, last_name from actor
  真实: SELECT actor_id, first_name, last_name FROM actor;
  结果:
    - 精确匹配: False
    - 标准化匹配: True
    - 关键词分数: 1.00

测试 3: 列名不同
  预测: SELECT * FROM actor;
  真实: SELECT actor_id, first_name, last_name FROM actor;
  结果:
    - 精确匹配: False
    - 标准化匹配: False
    - 关键词分数: 1.00

测试 4: 表名不同
  预测: SELECT * FROM film;
  真实: SELECT actor_id, first_name, last_name FROM actor;
  结果:
    - 精确匹配: False
    - 标准化匹配: False
    - 关键词分数: 0.50



### 2.7 完整评估流程

实现完整的 Text2SQL 评估流程，包括：
1. 数据集划分（训练集用于检索示例，测试集用于评估）
2. 对测试集的每个问题检索示例
3. 模拟生成 SQL（这里使用检索到的最相似示例的 SQL 作为预测）
4. 计算评估指标

In [9]:
from sklearn.model_selection import train_test_split

# 划分数据集（80% 训练，20% 测试）
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"训练集大小: {len(train_df)}")
print(f"测试集大小: {len(test_df)}")

# 重新构建向量库（只使用训练集）
eval_client = chromadb.Client()
eval_collection = eval_client.create_collection(
    name="text2sql_train",
    embedding_function=text2sql_ef
)

# 添加训练集示例
train_docs = []
train_metas = []
train_ids = []

for idx, row in train_df.iterrows():
    train_docs.append(row['question'])
    train_metas.append({
        'question': row['question'],
        'sql': row['sql']
    })
    train_ids.append(f"train_{idx}")

eval_collection.add(
    documents=train_docs,
    metadatas=train_metas,
    ids=train_ids
)

print(f"\n已添加 {len(train_docs)} 个训练示例到评估向量库")

训练集大小: 28
测试集大小: 8

已添加 28 个训练示例到评估向量库


In [10]:
def evaluate_text2sql_system(test_data, collection, use_top_k=1):
    """
    评估 Text2SQL 系统
    use_top_k: 使用检索到的第 k 个示例的 SQL 作为预测（模拟生成）
    """
    results = []
    
    for idx, row in test_data.iterrows():
        question = row['question']
        ground_truth_sql = row['sql']
        
        # 检索相似示例
        search_results = collection.query(
            query_texts=[question],
            n_results=use_top_k
        )
        
        # 使用最相似示例的 SQL 作为预测（模拟生成）
        predicted_sql = search_results['metadatas'][0][use_top_k-1]['sql']
        
        # 评估
        eval_result = evaluate_sql(predicted_sql, ground_truth_sql)
        eval_result['question'] = question
        eval_result['predicted'] = predicted_sql
        eval_result['ground_truth'] = ground_truth_sql
        eval_result['distance'] = search_results['distances'][0][use_top_k-1]
        
        results.append(eval_result)
    
    return results

# 运行评估
print("正在评估 Text2SQL 系统...\n")
eval_results = evaluate_text2sql_system(test_df, eval_collection, use_top_k=1)

# 计算整体指标
exact_match_acc = sum(r['exact_match'] for r in eval_results) / len(eval_results)
normalized_match_acc = sum(r['normalized_match'] for r in eval_results) / len(eval_results)
avg_keyword_score = sum(r['keyword_score'] for r in eval_results) / len(eval_results)

print("="*80)
print("评估结果汇总")
print("="*80)
print(f"测试样本数: {len(eval_results)}")
print(f"\n准确率指标:")
print(f"  - 精确匹配率: {exact_match_acc*100:.2f}%")
print(f"  - 标准化匹配率: {normalized_match_acc*100:.2f}%")
print(f"  - 平均关键词分数: {avg_keyword_score:.4f}")
print("="*80)

正在评估 Text2SQL 系统...

评估结果汇总
测试样本数: 8

准确率指标:
  - 精确匹配率: 0.00%
  - 标准化匹配率: 0.00%
  - 平均关键词分数: 0.4333


### 2.8 详细分析评估结果

分析哪些类型的查询表现较好，哪些较差。

In [11]:
# 展示一些评估案例
print("评估案例分析:\n")
print("="*80)

# 展示成功案例
print("\n✅ 成功案例（标准化匹配）:\n")
success_cases = [r for r in eval_results if r['normalized_match']]
for i, case in enumerate(success_cases[:3]):
    print(f"{i+1}. 问题: {case['question']}")
    print(f"   真实 SQL: {case['ground_truth']}")
    print(f"   预测 SQL: {case['predicted']}")
    print(f"   检索距离: {case['distance']:.4f}\n")

# 展示失败案例
print("\n❌ 失败案例（未匹配）:\n")
failure_cases = [r for r in eval_results if not r['normalized_match']]
for i, case in enumerate(failure_cases[:3]):
    print(f"{i+1}. 问题: {case['question']}")
    print(f"   真实 SQL: {case['ground_truth']}")
    print(f"   预测 SQL: {case['predicted']}")
    print(f"   关键词分数: {case['keyword_score']:.4f}")
    print(f"   检索距离: {case['distance']:.4f}\n")

print("="*80)

评估案例分析:


✅ 成功案例（标准化匹配）:


❌ 失败案例（未匹配）:

1. 问题: Close (delete) store with ID 3.
   真实 SQL: DELETE FROM store WHERE store_id = 3;
   预测 SQL: DELETE FROM inventory WHERE inventory_id = 21;
   关键词分数: 0.6000
   检索距离: 0.7824

2. 问题: Create a new customer for store 1 named 'Alice Brown'.
   真实 SQL: INSERT INTO customer (store_id, first_name, last_name, create_date, address_id, active) VALUES (1, 'Alice', 'Brown', NOW(), 1, 1);
   预测 SQL: INSERT INTO staff (first_name, last_name, address_id, store_id, active, username) VALUES ('Bob', 'Lee', 1, 1, 1, 'boblee');
   关键词分数: 0.3333
   检索距离: 0.6944

3. 问题: Change payment amount of payment ID 6 to 12.50.
   真实 SQL: UPDATE payment SET amount = 12.50 WHERE payment_id = 6;
   预测 SQL: DELETE FROM payment WHERE payment_id = 7;
   关键词分数: 0.4000
   检索距离: 0.8397



### 2.9 按SQL类型分析性能

In [12]:
# 按 SQL 类型分析性能
print("按 SQL 类型的性能分析:\n")
print("="*80)

sql_type_performance = {}
for result in eval_results:
    # 提取 SQL 类型
    sql_type = result['ground_truth'].split()[0].upper()
    
    if sql_type not in sql_type_performance:
        sql_type_performance[sql_type] = {
            'count': 0,
            'exact_match': 0,
            'normalized_match': 0,
            'keyword_scores': []
        }
    
    sql_type_performance[sql_type]['count'] += 1
    sql_type_performance[sql_type]['exact_match'] += result['exact_match']
    sql_type_performance[sql_type]['normalized_match'] += result['normalized_match']
    sql_type_performance[sql_type]['keyword_scores'].append(result['keyword_score'])

for sql_type, perf in sql_type_performance.items():
    count = perf['count']
    if count == 0:
        continue
    exact_acc = perf['exact_match'] / count * 100
    norm_acc = perf['normalized_match'] / count * 100
    avg_keyword = sum(perf['keyword_scores']) / count
    
    print(f"{sql_type} 类型 (样本数: {count}):")
    print(f"  - 精确匹配率: {exact_acc:.1f}%")
    print(f"  - 标准化匹配率: {norm_acc:.1f}%")
    print(f"  - 平均关键词分数: {avg_keyword:.4f}\n")

print("="*80)

按 SQL 类型的性能分析:

DELETE 类型 (样本数: 2):
  - 精确匹配率: 0.0%
  - 标准化匹配率: 0.0%
  - 平均关键词分数: 0.6000

INSERT 类型 (样本数: 2):
  - 精确匹配率: 0.0%
  - 标准化匹配率: 0.0%
  - 平均关键词分数: 0.3333

UPDATE 类型 (样本数: 2):
  - 精确匹配率: 0.0%
  - 标准化匹配率: 0.0%
  - 平均关键词分数: 0.4500

SELECT 类型 (样本数: 2):
  - 精确匹配率: 0.0%
  - 标准化匹配率: 0.0%
  - 平均关键词分数: 0.3500



### 2.10 总结与改进方向

**当前系统特点：**
- 使用向量检索选择最相似的示例
- 直接使用检索到的示例 SQL 作为预测（模拟零样本预测）

**评估体系的优势：**
1. ✅ 多维度评估指标（精确匹配、标准化匹配、关键词匹配）
2. ✅ 按 SQL 类型细分的性能分析
3. ✅ 详细的案例分析（成功/失败案例）

**改进方向：**
1. **集成真实 LLM**: 使用 GPT-4/Claude 等模型基于检索到的示例生成 SQL
2. **增强示例检索**: 
   - 使用重排序模型提高检索质量
   - 考虑 SQL 结构相似性（不仅仅是问题相似性）
3. **数据库 Schema 增强**: 将表结构信息加入 Prompt
4. **执行验证**: 在测试数据库上实际执行 SQL，验证结果正确性
5. **多样性采样**: 检索多个不同类型的示例提供更全面的上下文
6. **错误分析**: 构建错误类型分类器，识别常见错误模式

**实际应用建议：**
- 对于生产环境，建议标准化匹配率达到 80% 以上
- 关键词分数可用于识别部分正确的查询
- 结合人工审核处理低置信度的预测

---

## 🎉 作业完成总结

本次作业我们完成了两个重要任务：

### 作业1：检索结果后处理方法 ✅
- **重排技术**: Cross-Encoder 重排、MMR 多样性重排
- **压缩技术**: 相关性过滤、上下文压缩、文本摘要
- **校正技术**: 查询扩展、查询重写、混合检索
- **综合应用**: 完整检索管道、方法对比

### 作业2：Text2SQL 评估体系 ✅
- **数据处理**: 加载和分析 Sakila 数据集
- **向量检索**: 构建 Few-Shot 示例检索系统
- **评估指标**: 精确匹配、标准化匹配、关键词匹配
- **完整评估**: 数据集划分、评估流程、结果分析

### 核心收获
1. 掌握了多种检索后处理技术及其组合使用
2. 了解了 Text2SQL 任务的评估方法和关键指标
3. 学会了构建完整的评估流程和性能分析体系
4. 理解了向量检索在Few-Shot学习中的应用